# Scratchpad

This is just for me to experiment with stuff. Don't pay attention.

## OpenEMS Tutorials

I want to run through the tutorials a bit before I dive head first into the deep end.

### Waveguide

From: https://openems.readthedocs.io/en/latest/python/openEMS/Tutorials/Rect_Waveguide.html

In [ ]:
import os, tempfile
from pylab import *

from CSXCAD  import ContinuousStructure
from openEMS import openEMS
from openEMS.physical_constants import *

original_path = os.getcwd()
sim_path = os.path.join(os.getcwd(), 'horn_antenna_sim/Rect_WG')
if not os.path.exists(sim_path):
    os.mkdir(sim_path)

post_proc_only = False
unit = 1e-6 #drawing unit in micrometers

# waveguide dimensions
# WR42
a = 10700   #waveguide width
b = 4300    #waveguide height
length = 50000

# frequency range of interest
f_start = 20e9
f_0     = 24e9
f_stop  = 26e9
lambda0 = C0/f_0/unit

#waveguide TE-mode definition
TE_mode = 'TE10'

#targeted mesh resolution
mesh_res = lambda0/30

# Setup FDTD parameter & excitation function
FDTD = openEMS(NrTS=1e4)
FDTD.SetGaussExcite(0.5*(f_start+f_stop),0.5*(f_stop-f_start))

# boundary conditions
FDTD.SetBoundaryCond([0, 0, 0, 0, 3, 3])

# Setup geometry & mesh
CSX = ContinuousStructure()
FDTD.SetCSX(CSX)
mesh = CSX.GetGrid()
mesh.SetDeltaUnit(unit)

mesh.AddLine('x', [0, a])
mesh.AddLine('y', [0, b])
mesh.AddLine('z', [0, length])

# Apply the waveguide port
ports = []
start=[0, 0, 10*mesh_res]
stop =[a, b, 15*mesh_res]
mesh.AddLine('z', [start[2], stop[2]])
ports.append(FDTD.AddRectWaveGuidePort( 0, start, stop, 'z', a*unit, b*unit, TE_mode, 1))

start=[0, 0, length-10*mesh_res]
stop =[a, b, length-15*mesh_res]
mesh.AddLine('z', [start[2], stop[2]])
ports.append(FDTD.AddRectWaveGuidePort( 1, start, stop, 'z', a*unit, b*unit, TE_mode))

mesh.SmoothMeshLines('all', mesh_res, ratio=1.4)

# Define dump box…
Et = CSX.AddDump('Et', file_type=0, sub_sampling=[2,2,2])
start = [0, 0, 0]
stop  = [a, b, length]
Et.AddBox(start, stop)

# Save the mesh to file
CSX_file = os.path.join(sim_path, 'debug_mesh_rect_wg.xml')
CSX.Write2XML(CSX_file)
print(f"Mesh saved to: {CSX_file}")


In [ ]:
# Run the simulation
if 0:  # debugging only
    CSX_file = os.path.join(sim_path, 'rect_wg.xml')
    if not os.path.exists(sim_path):
        os.mkdir(sim_path)
    CSX.Write2XML(CSX_file)
    from CSXCAD import AppCSXCAD_BIN
    os.system(AppCSXCAD_BIN + ' "{}"'.format(CSX_file))

if not post_proc_only:
    FDTD.Run(sim_path, cleanup=True)

os.chdir(original_path)

In [ ]:
# Postprocessing & plotting
freq = linspace(f_start,f_stop,201)
for port in ports:
    port.CalcPort(sim_path, freq)

s11 = ports[0].uf_ref / ports[0].uf_inc
s21 = ports[1].uf_ref / ports[0].uf_inc
ZL  = ports[0].uf_tot / ports[0].if_tot
ZL_a = ports[0].ZL # analytic waveguide impedance

# Plot S-parameters
figure()
plot(freq*1e-6,20*log10(abs(s11)),'k-',linewidth=2, label=r'$S_{11}$')
grid()
plot(freq*1e-6,20*log10(abs(s21)),'r--',linewidth=2, label=r'$S_{21}$')
legend();
ylabel('S-Parameter (dB)')
xlabel(r'frequency (MHz) $\rightarrow$')

# Compare analytic and numerical wave-impedance
figure()
plot(freq*1e-6,real(ZL), linewidth=2, label=r'$\Re\{Z_L\}$')
grid()
plot(freq*1e-6,imag(ZL),'r--', linewidth=2, label=r'$\Im\{Z_L\}$')
plot(freq*1e-6,ZL_a,'g-.',linewidth=2, label=r'$Z_{L, analytic}$')
ylabel(r'ZL $(\Omega)$')
xlabel(r'frequency (MHz) $\rightarrow$')
legend()

show()

### Patch Antenna

From https://openems.readthedocs.io/en/latest/python/openEMS/Tutorials/Simple_Patch_Antenna.html

In [ ]:
import os, tempfile
from pylab import *

from CSXCAD  import ContinuousStructure
from openEMS import openEMS
from openEMS.physical_constants import *

# Parameter Setup
original_path = os.getcwd()
Sim_Path = os.path.join(os.getcwd(), 'horn_antenna_sim/Simp_Patch')
if not os.path.exists(Sim_Path):
    os.mkdir(Sim_Path)

post_proc_only = False

# patch width (resonant length) in x-direction
patch_width  = 32 #
# patch length in y-direction
patch_length = 40

#substrate setup
substrate_epsR   = 3.38
substrate_kappa  = 1e-3 * 2*pi*2.45e9 * EPS0*substrate_epsR
substrate_width  = 60
substrate_length = 60
substrate_thickness = 1.524
substrate_cells = 4

#setup feeding
feed_pos = -6 #feeding position in x-direction
feed_R = 50     #feed resistance

# size of the simulation box
SimBox = np.array([200, 200, 150])

# setup FDTD parameter & excitation function
f0 = 2e9 # center frequency
fc = 1e9 # 20 dB corner frequency

# FDTD Setup
FDTD = openEMS(NrTS=30000, EndCriteria=1e-4)
FDTD.SetGaussExcite( f0, fc )
FDTD.SetBoundaryCond( ['MUR', 'MUR', 'MUR', 'MUR', 'MUR', 'MUR'] )


CSX = ContinuousStructure()
FDTD.SetCSX(CSX)
mesh = CSX.GetGrid()
mesh.SetDeltaUnit(1e-3)
mesh_res = C0/(f0+fc)/1e-3/20


# Create mesh n' stuff
#initialize the mesh with the "air-box" dimensions
mesh.AddLine('x', [-SimBox[0]/2, SimBox[0]/2])
mesh.AddLine('y', [-SimBox[1]/2, SimBox[1]/2]          )
mesh.AddLine('z', [-SimBox[2]/3, SimBox[2]*2/3]        )

# create patch
patch = CSX.AddMetal( 'patch' ) # create a perfect electric conductor (PEC)
start = [-patch_width/2, -patch_length/2, substrate_thickness]
stop  = [ patch_width/2 , patch_length/2, substrate_thickness]
patch.AddBox(priority=10, start=start, stop=stop) # add a box-primitive to the metal property 'patch'
FDTD.AddEdges2Grid(dirs='xy', properties=patch, metal_edge_res=mesh_res/2)

# create substrate
substrate = CSX.AddMaterial( 'substrate', epsilon=substrate_epsR, kappa=substrate_kappa)
start = [-substrate_width/2, -substrate_length/2, 0]
stop  = [ substrate_width/2,  substrate_length/2, substrate_thickness]
substrate.AddBox( priority=0, start=start, stop=stop )

# add extra cells to discretize the substrate thickness
mesh.AddLine('z', linspace(0,substrate_thickness,substrate_cells+1))

# create ground (same size as substrate)
gnd = CSX.AddMetal( 'gnd' ) # create a perfect electric conductor (PEC)
start[2]=0
stop[2] =0
gnd.AddBox(start, stop, priority=10)

FDTD.AddEdges2Grid(dirs='xy', properties=gnd)

# apply the excitation & resist as a current source
start = [feed_pos, 0, 0]
stop  = [feed_pos, 0, substrate_thickness]
port = FDTD.AddLumpedPort(1, feed_R, start, stop, 'z', 1.0, priority=5, edges2grid='xy')

mesh.SmoothMeshLines('all', mesh_res, 1.4)

# Add the nf2ff recording box
nf2ff = FDTD.CreateNF2FFBox()

# Save the mesh to file
CSX_file = os.path.join(os.getcwd(), 'horn_antenna_sim/simp_patch.xml')
CSX.Write2XML(CSX_file)
print(f"Mesh saved to: {CSX_file}")


In [ ]:
# Run Simulation

if 0:  # debugging only
    CSX_file = os.path.join(Sim_Path, 'simp_patch.xml')
    if not os.path.exists(Sim_Path):
        os.mkdir(Sim_Path)
    CSX.Write2XML(CSX_file)
    from CSXCAD import AppCSXCAD_BIN
    os.system(AppCSXCAD_BIN + ' "{}"'.format(CSX_file))

if not post_proc_only:
    FDTD.Run(Sim_Path, cleanup=True)

os.chdir(original_path)


### Conical Horn Antenna

From: https://github.com/thliebig/openEMS/blob/master/matlab/Tutorials/Conical_Horn_Antenna.m

Original Matlab Code:

```matlab
%
% Tutorials / conical horn antenna
%
% Description at:
% http://openems.de/index.php/Tutorial:_Conical_Horn_Antenna
%
% Tested with
%  - Matlab 2011a / Octave 4.0
%  - openEMS v0.0.33
%
% (C) 2011-2015 Thorsten Liebig <thorsten.liebig@uni-due.de>

close all
clear
clc

%% setup the simulation
physical_constants;
unit = 1e-3; % all length in mm

% horn radius
horn.radius  = 20;
% horn length in z-direction
horn.length = 50;

horn.feed_length = 50;

horn.thickness = 2;

% horn opening angle
horn.angle = 20*pi/180;

% size of the simulation box
SimBox = [100 100 100]*2;

% frequency range of interest
f_start =  10e9;
f_stop  =  20e9;

% frequency of interest
f0 = 15e9;

%% setup FDTD parameter & excitation function
FDTD = InitFDTD( 'NrTS', 30000, 'EndCriteria', 1e-4 );
FDTD = SetGaussExcite(FDTD,0.5*(f_start+f_stop),0.5*(f_stop-f_start));
BC = {'PML_8' 'PML_8' 'PML_8' 'PML_8' 'PML_8' 'PML_8'}; % boundary conditions
FDTD = SetBoundaryCond( FDTD, BC );

%% setup CSXCAD geometry & mesh
% currently, openEMS cannot automatically generate a mesh
max_res = c0 / (f_stop) / unit / 15; % cell size: lambda/20
CSX = InitCSX();

%create fixed lines for the simulation box, substrate and port
mesh.x = [-SimBox(1)/2 -horn.radius 0 horn.radius SimBox(1)/2];
mesh.x = SmoothMeshLines( mesh.x, max_res, 1.4); % create a smooth mesh between specified fixed mesh lines

mesh.y = mesh.x;

%create fixed lines for the simulation box and given number of lines inside the substrate
mesh.z = [-horn.feed_length 0 SimBox(3) ];
mesh.z = SmoothMeshLines( mesh.z, max_res, 1.4 );

CSX = DefineRectGrid( CSX, unit, mesh );

%% create horn
% horn + waveguide, defined by a rotational polygon
CSX = AddMetal(CSX, 'Conical_Horn');
p(1,1) = horn.radius+horn.thickness;   % x-coord point 1
p(2,1) = -horn.feed_length;     % z-coord point 1
p(1,end+1) = horn.radius+horn.thickness;   % x-coord point 1
p(2,end) = 0;     % z-coord point 1
p(1,end+1) = horn.radius+horn.thickness + sin(horn.angle)*horn.length; % x-coord point 2
p(2,end) = horn.length; % y-coord point 2
p(1,end+1) = horn.radius + sin(horn.angle)*horn.length; % x-coord point 2
p(2,end) = horn.length; % y-coord point 2
p(1,end+1) = horn.radius;  % x-coord point 1
p(2,end) = 0;     % z-coord point 1
p(1,end+1) = horn.radius;   % x-coord point 1
p(2,end) = -horn.feed_length;     % z-coord point 1
CSX = AddRotPoly(CSX,'Conical_Horn',10,'x',p,'z');

% horn aperture
A = pi*((horn.radius + sin(horn.angle)*horn.length)*unit)^2;

%% apply the excitation %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
start=[-horn.radius -horn.radius mesh.z(10) ];
stop =[+horn.radius +horn.radius mesh.z(1)+horn.feed_length/2 ];
[CSX, port] = AddCircWaveGuidePort( CSX, 0, 1, start, stop, horn.radius*unit, 'TE11', 0, 1);

%%
CSX = AddDump(CSX,'Exc_dump');
start=[-horn.radius -horn.radius mesh.z(8)];
stop =[+horn.radius +horn.radius mesh.z(8)];
CSX = AddBox(CSX,'Exc_dump',0,start,stop);

%% nf2ff calc
start = [mesh.x(9) mesh.y(9) mesh.z(9)];
stop  = [mesh.x(end-8) mesh.y(end-8) mesh.z(end-8)];
[CSX nf2ff] = CreateNF2FFBox(CSX, 'nf2ff', start, stop, 'Directions', [1 1 1 1 0 1]);

%% prepare simulation folder
Sim_Path = 'tmp';
Sim_CSX = 'horn_ant.xml';

[status, message, messageid] = rmdir( Sim_Path, 's' ); % clear previous directory
[status, message, messageid] = mkdir( Sim_Path ); % create empty simulation folder

%% write openEMS compatible xml-file
WriteOpenEMS( [Sim_Path '/' Sim_CSX], FDTD, CSX );

%% show the structure
CSXGeomPlot( [Sim_Path '/' Sim_CSX] );

%% run openEMS
RunOpenEMS( Sim_Path, Sim_CSX);

%% postprocessing & do the plots
freq = linspace(f_start,f_stop,201);

port = calcPort(port, Sim_Path, freq);

Zin = port.uf.tot ./ port.if.tot;
s11 = port.uf.ref ./ port.uf.inc;

% plot reflection coefficient S11
figure
plot( freq/1e9, 20*log10(abs(s11)), 'k-', 'Linewidth', 2 );
ylim([-60 0]);
grid on
title( 'reflection coefficient S_{11}' );
xlabel( 'frequency f / GHz' );
ylabel( 'reflection coefficient |S_{11}|' );

drawnow

%% NFFF contour plots %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% calculate the far field at phi=0 degrees and at phi=90 degrees
thetaRange = (0:2:359) - 180;
disp( 'calculating far field at phi=[0 90] deg...' );
nf2ff = CalcNF2FF(nf2ff, Sim_Path, f0, thetaRange*pi/180, [0 90]*pi/180);

Dlog=10*log10(nf2ff.Dmax);
G_a = 4*pi*A/(c0/f0)^2;
e_a = nf2ff.Dmax/G_a;

% display some antenna parameter
disp( ['radiated power: Prad = ' num2str(nf2ff.Prad) ' Watt']);
disp( ['directivity: Dmax = ' num2str(Dlog) ' dBi'] );
disp( ['aperture efficiency: e_a = ' num2str(e_a*100) '%'] );

%%
% normalized directivity
figure
plotFFdB(nf2ff,'xaxis','theta','param',[1 2]);
drawnow
%   D_log = 20*log10(nf2ff.E_norm{1}/max(max(nf2ff.E_norm{1})));
%   D_log = D_log + 10*log10(nf2ff.Dmax);
%   plot( nf2ff.theta, D_log(:,1) ,'k-', nf2ff.theta, D_log(:,2) ,'r-' );

% polar plot
figure
polarFF(nf2ff,'xaxis','theta','param',[1 2],'logscale',[-40 20], 'xtics', 12);
drawnow
%   polar( nf2ff.theta, nf2ff.E_norm{1}(:,1) )

%% calculate 3D pattern
phiRange = sort( unique( [-180:5:-100 -100:2.5:-50 -50:1:50 50:2.5:100 100:5:180] ) );
thetaRange = sort( unique([ 0:1:50 50:2.:100 100:5:180 ]));

disp( 'calculating 3D far field...' );
nf2ff = CalcNF2FF(nf2ff, Sim_Path, f0, thetaRange*pi/180, phiRange*pi/180, 'Verbose',2,'Outfile','nf2ff_3D.h5');

figure
plotFF3D(nf2ff);        % plot liear 3D far field

%%
E_far_normalized = nf2ff.E_norm{1}/max(nf2ff.E_norm{1}(:));
DumpFF2VTK([Sim_Path '/Conical_Horn_Pattern.vtk'],E_far_normalized,thetaRange,phiRange,'scale',1e-3);
```

In [ ]:
import os
from pylab import *
from CSXCAD import ContinuousStructure
from openEMS import openEMS
from openEMS.physical_constants import *
from scipy.special import jn, jn_zeros

# Parameter Setup
original_path = os.getcwd()
Sim_Path = os.path.join(os.getcwd(), 'horn_antenna_sim/Conical_Horn_Antenna')
if not os.path.exists(Sim_Path):
    os.mkdir(Sim_Path)

post_proc_only = False

# setup the simulation
unit = 1e-3  # all length in mm

# horn radius
horn_radius = 20
# horn length in z-direction
horn_length = 50
horn_feed_length = 50
horn_thickness = 2

# horn opening angle
horn_angle = 20*pi/180

# size of the simulation box
SimBox = array([100, 100, 100]) * 2

# frequency range of interest
f_start = 10e9
f_stop = 20e9

# frequency of interest
f0 = 15e9

# setup FDTD parameter & excitation function
FDTD = openEMS(NrTS=30000, EndCriteria=1e-4)
FDTD.SetGaussExcite(0.5*(f_start+f_stop), 0.5*(f_stop-f_start))
BC = ['PML_8', 'PML_8', 'PML_8', 'PML_8', 'PML_8', 'PML_8']
FDTD.SetBoundaryCond(BC)

# setup CSXCAD geometry & mesh
max_res = C0 / f_stop / unit / 15  # cell size: lambda/15
CSX = ContinuousStructure()
FDTD.SetCSX(CSX)
mesh = CSX.GetGrid()
mesh.SetDeltaUnit(unit)

# create fixed lines for the simulation box
mesh.AddLine('x', [-SimBox[0]/2, -horn_radius, 0, horn_radius, SimBox[0]/2])
mesh.AddLine('y', [-SimBox[1]/2, -horn_radius, 0, horn_radius, SimBox[1]/2])
mesh.AddLine('z', [-horn_feed_length, 0, SimBox[2]])

# smooth the mesh
mesh.SmoothMeshLines('all', max_res, 1.4)

# create horn - defined by a rotational polygon
horn = CSX.AddMetal('Conical_Horn')
p = zeros([2, 6])
p[0, 0] = horn_radius + horn_thickness
p[1, 0] = -horn_feed_length
p[0, 1] = horn_radius + horn_thickness
p[1, 1] = 0
p[0, 2] = horn_radius + horn_thickness + sin(horn_angle)*horn_length
p[1, 2] = horn_length
p[0, 3] = horn_radius + sin(horn_angle)*horn_length
p[1, 3] = horn_length
p[0, 4] = horn_radius
p[1, 4] = 0
p[0, 5] = horn_radius
p[1, 5] = -horn_feed_length

# Points, elevation, rot_axes, and rotation angle
horn.AddRotPoly(p, "x", 10.0, "y", [0.0, 2*pi])

# horn aperture
A = pi * ((horn_radius + sin(horn_angle)*horn_length) * unit)**2

# apply the excitation - TE11 mode in circular waveguide
start = [-horn_radius, -horn_radius, -horn_feed_length]
stop = [horn_radius, horn_radius, -horn_feed_length/2]

# For TE11 mode: cutoff wavenumber kc = p'_11 / a, where p'_11 is first root of J'_1(x) = 0
# p'_11 ≈ 1.841
p_11 = 1.841
kc = p_11 / (horn_radius * unit)

# TE11 mode field functions for circular waveguide (oriented along y-axis)
# E_func and H_func must be arrays of 3 strings [E_x, E_y, E_z] and [H_x, H_y, H_z]
r_expr = f'sqrt(x^2+y^2)/{horn_radius*unit}'
bessel_expr = f'JnBessel(1, {p_11}*{r_expr})'

# For TE11 mode with E-field polarized in y-direction:
E_func = ['0', bessel_expr, '0']  # [E_x, E_y, E_z]
H_func = [bessel_expr, '0', '0']  # [H_x, H_y, H_z]

port = FDTD.AddWaveGuidePort(0, start, stop, 'z', E_func, H_func, kc, excite=1)

# nf2ff calc
nf2ff = FDTD.CreateNF2FFBox()

# write openEMS compatible xml-file
CSX_file = os.path.join(os.getcwd(), 'horn_antenna_sim/test_conical_horn.xml')
CSX.Write2XML(CSX_file)
print(f"Mesh saved to: {CSX_file}")

FileNotFoundError: Directory in which file is to be saved does not exist. 

In [ ]:
# Run Simulation
os.chdir(original_path)